# Sarimax Forecasting

In [1]:
import pandas as pd

### 1) LOAD TRAFFIC COUNTER DATA, KEEP ONLY “Kfz” (all motor vehicles)

In [2]:
traffic = pd.read_csv("../data/dauerzaehlstellen_data.csv", parse_dates=["DATUM"])
traffic = (
    traffic
    .query('FZTYP=="Kfz" and RINAME=="Gesamt"')
    .rename(columns={"DATUM":"DATE"})[["ZNR","DATE","DTVMS","ISTCOVID19"]]
)

### 2) LOAD LOCATION FILE TO GET DISTRICT_CODE FOR EACH ZNR

In [3]:
loc = pd.read_csv("../data/dauerzaehlstellen_location.csv")
loc = loc.rename(columns={"BEZIRK_PLZ":"BEZIRK"})[["ZNR","BEZIRK"]]
traffic = traffic.merge(loc, on="ZNR", how="left")
traffic = traffic[traffic["ZNR"] != 1185]  # drop bad counter


### 3) LOAD EXOGENOUS DATASETS (all monthly)

In [4]:
ausp = pd.read_csv("data_arima_final/auspendler_by_bezirk.csv", parse_dates=["DATE"])
ausp["AUSPENDLER"] = ausp["AUSPENDLER"].astype(float)

pop = pd.read_csv("data_arima_final/population_by_bezirk.csv", parse_dates=["DATE"])
pop["POP"] = pop["POP"].astype(float)

veh = pd.read_csv("data_arima_final/vehicle_density.csv", parse_dates=["DATE"])
veh["PKW_DENSITY"] = veh["PKW_DENSITY"].astype(float)

vmw = pd.read_csv("data_arima_final/verkehrsmittelwahl.csv", parse_dates=["DATE"])
for col in ["BICYCLE","BIKESHARING","BY_FOOT","CAR","CARSHARING","MOTORBIKE","PUBLIC_TRANSPORT"]:
    vmw[col] = vmw[col].astype(float)

### 4) BUILD FULL PANEL UNTIL 2030

In [5]:
# Build the full panel (ZNR × every month from first traffic → Dec 2030)
znrs      = traffic["ZNR"].unique()
start     = traffic["DATE"].min().to_period("M").to_timestamp()
end       = pd.to_datetime("2030-12-01")
all_months= pd.date_range(start, end, freq="MS")

panel = (
    pd.MultiIndex
      .from_product([znrs, all_months], names=["ZNR","DATE"])
      .to_frame(index=False)
)

# Merge in the observed counts & BEZIRK
panel = panel.merge(traffic, on=["ZNR","DATE"], how="left")

# propagate each ZNR’s BEZIRK into all its months
panel["BEZIRK"] = (
    panel
    .groupby("ZNR")["BEZIRK"]
    .transform(lambda s: s.ffill().bfill())
)

# mark all non‐observed months as no‐COVID
panel["ISTCOVID19"] = panel["ISTCOVID19"].fillna(0)

In [6]:
panel.tail(2)

,ZNR,DATE,DTVMS,ISTCOVID19,BEZIRK
11158,1197,2030-11-01,NaN,0.0,1220.0
11159,1197,2030-12-01,NaN,0.0,1220.0


### 4) MERGE TRAFFIC + EXOGENOUS ON [DATE, BEZIRK] (AND DATE for vmw)

In [7]:
# Now merge all exogenous variables
panel = panel.merge(ausp, on=["DATE","BEZIRK"], how="left")
panel = panel.merge(pop,  on=["DATE","BEZIRK"], how="left")
panel = panel.merge(veh[["DATE","BEZIRK","PKW_DENSITY"]], on=["DATE","BEZIRK"], how="left")
panel = panel.merge(vmw, on="DATE", how="left")

In [8]:
panel = panel.sort_values(["ZNR","DATE"]).reset_index(drop=True)

In [9]:
panel.head(5)

,ZNR,DATE,DTVMS,ISTCOVID19,BEZIRK,AUSPENDLER,POP,PKW_DENSITY,BICYCLE,BIKESHARING,BY_FOOT,CAR,CARSHARING,MOTORBIKE,PUBLIC_TRANSPORT
0,1075,2016-01-01,33645.0,0.0,1220.0,0.105,180272.0,433.11,0.069791,0.000997,0.270189,0.269192,0.001994,0.002991,0.384845
1,1075,2016-02-01,36981.0,0.0,1220.0,0.105,180272.0,433.11,0.069791,0.000997,0.270189,0.269192,0.001994,0.002991,0.384845
2,1075,2016-03-01,39057.0,0.0,1220.0,0.105,180272.0,433.11,0.069791,0.000997,0.270189,0.269192,0.001994,0.002991,0.384845
3,1075,2016-04-01,41570.0,0.0,1220.0,0.105,180272.0,433.11,0.069791,0.000997,0.270189,0.269192,0.001994,0.002991,0.384845
4,1075,2016-05-01,37195.0,0.0,1220.0,0.105,180272.0,433.11,0.069791,0.000997,0.270189,0.269192,0.001994,0.002991,0.384845


### 5) CHECK MISSING

In [10]:
# number of ZNR × number of months expected
n_z = len(znrs)
n_m = len(all_months)
print(f"Expecting {n_z}×{n_m} = {n_z * n_m} rows")

# actual
print("Panel rows:", len(panel))

# should be equal
assert len(panel) == n_z * n_m, "⚠️ panel is missing some (ZNR,DATE) combinations!"

Expecting 62×180 = 11160 rows
Panel rows: 11160


In [11]:
# any missing district codes?
print("Missing BEZIRK:", panel["BEZIRK"].isna().sum())

# any missing covid dummy?
print("Missing ISTCOVID19:", panel["ISTCOVID19"].isna().sum())

Missing BEZIRK: 0
Missing ISTCOVID19: 0


In [12]:
# list all exog columns
exog_cols = ["AUSPENDLER","POP","PKW_DENSITY"] + \
            ["BICYCLE","BIKESHARING","BY_FOOT","CAR","CARSHARING","MOTORBIKE","PUBLIC_TRANSPORT"]

miss = panel[exog_cols].isna().sum()
print("Missing values per exog column:\n", miss)

Missing values per exog column:
 AUSPENDLER          0
POP                 0
PKW_DENSITY         0
BICYCLE             0
BIKESHARING         0
BY_FOOT             0
CAR                 0
CARSHARING          0
MOTORBIKE           0
PUBLIC_TRANSPORT    0
dtype: int64


In [13]:
# for each counter, count how many observed months you actually have
obs_counts = panel.groupby("ZNR")["DTVMS"].apply(lambda s: s.notna().sum())

# expected obs span (e.g. if first obs per counter is 2016-01 to last obs 2024-12 → 108 months)
print(obs_counts.describe())

# counters with fewer than, say, 36 obs you might skip or treat specially
print("Short histories:", obs_counts[obs_counts < 36].index.tolist())

count     62.000000
mean     103.258065
std       11.192702
min       63.000000
25%      108.000000
50%      108.000000
75%      108.000000
max      108.000000
Name: DTVMS, dtype: float64
Short histories: []


In [14]:
# count non-NaN DTVMS per ZNR
obs_counts = panel.groupby("ZNR")["DTVMS"].apply(lambda s: s.notna().sum())

print(obs_counts.describe())
print("Counters with < 36 months of history:", obs_counts[obs_counts<36].index.tolist())

count     62.000000
mean     103.258065
std       11.192702
min       63.000000
25%      108.000000
50%      108.000000
75%      108.000000
max      108.000000
Name: DTVMS, dtype: float64
Counters with < 36 months of history: []


In [15]:
import pandas as pd

# assume `panel` is your DataFrame with columns ["ZNR","DATE","DTVMS",…]
# and DATE is a datetime64 column

def check_full_history(df, end_date="2024-12-01"):
    first_obs = df.loc[df["DTVMS"].notna(), "DATE"].min()
    if pd.isna(first_obs):
        return False, None, None, None
    # build the expected monthly dates
    expected = pd.date_range(first_obs, end_date, freq="MS")
    # actual months where DTVMS is observed
    actual   = df.loc[df["DTVMS"].notna(), "DATE"].drop_duplicates().sort_values()
    # count how many of the expected appear in actual
    n_expected = len(expected)
    n_actual   = actual.isin(expected).sum()
    missing    = n_expected - n_actual
    return missing == 0, first_obs, n_expected, missing

# apply per counter
results = []
for znr, grp in panel.groupby("ZNR"):
    complete, first, total_mons, n_missing = check_full_history(grp, "2024-12-01")
    results.append({
        "ZNR": znr,
        "first_obs": first,
        "months_expected": total_mons,
        "months_missing": n_missing,
        "full_history": complete
    })

check_df = pd.DataFrame(results)
print(check_df)

# List counters with a perfect history:
perfect = check_df[check_df["full_history"]]["ZNR"].tolist()
print("Counters with no gaps:", perfect)

# List counters with any gaps:
gappy = check_df[check_df["months_missing"]>0]
print("Counters with gaps:\n", gappy)

     ZNR  first_obs  months_expected  months_missing  full_history
0   1075 2016-01-01              108               0          True
1   1078 2016-01-01              108               0          True
2   1089 2016-01-01              108               0          True
3   1096 2016-01-01              108               0          True
4   1131 2016-01-01              108               0          True
..   ...        ...              ...             ...           ...
57  1623 2016-01-01              108               0          True
58  1624 2016-01-01              108               0          True
59  1625 2018-01-01               84               0          True
60  1626 2018-01-01               84               0          True
61  1627 2018-01-01               84               0          True

[62 rows x 5 columns]
Counters with no gaps: [1075, 1078, 1089, 1096, 1131, 1170, 1177, 1179, 1180, 1181, 1182, 1184, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1198, 1200, 1201,

In [16]:
import pandas as pd

# assume `panel` is your DataFrame with columns ["ZNR","DATE","DTVMS",…]
# and DATE is a datetime64 column

def check_full_history(df, end_date="2024-12-01"):
    obs = df["DTVMS"] > 0
    first_obs = df.loc[obs, "DATE"].min()
    expected  = pd.date_range(first_obs, end_date, freq="MS")
    actual    = df.loc[obs, "DATE"].drop_duplicates().sort_values()
    n_expected = len(expected)
    n_actual   = actual.isin(expected).sum()
    missing    = n_expected - n_actual
    return missing == 0, first_obs, n_expected, missing

# apply per counter
results = []
for znr, grp in panel.groupby("ZNR"):
    complete, first, total_mons, n_missing = check_full_history(grp, "2024-12-01")
    results.append({
        "ZNR": znr,
        "first_obs": first,
        "months_expected": total_mons,
        "months_missing": n_missing,
        "full_history": complete
    })

check_df = pd.DataFrame(results)
print(check_df)

# List counters with a perfect history:
perfect = check_df[check_df["full_history"]]["ZNR"].tolist()
print("Counters with no gaps:", perfect)

# List counters with any gaps:
gappy = check_df[check_df["months_missing"]>0]
print("Counters with gaps:\n", gappy)

     ZNR  first_obs  months_expected  months_missing  full_history
0   1075 2016-01-01              108               0          True
1   1078 2016-01-01              108               0          True
2   1089 2016-01-01              108               0          True
3   1096 2016-01-01              108               0          True
4   1131 2016-01-01              108               0          True
..   ...        ...              ...             ...           ...
57  1623 2016-01-01              108               0          True
58  1624 2016-01-01              108               0          True
59  1625 2018-01-01               84               0          True
60  1626 2018-01-01               84               0          True
61  1627 2018-01-01               84               0          True

[62 rows x 5 columns]
Counters with no gaps: [1075, 1078, 1089, 1096, 1131, 1170, 1177, 1179, 1180, 1181, 1182, 1184, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1198, 1200, 1201,

In [17]:
# 1) Drop the counter with too many gaps
panel = panel[panel["ZNR"] != 1197].copy()

# 2) List of counters to impute
to_impute = [1199, 1219, 1618]

# 3) Impute each one on its historical window (up to 2024-12-01)
for znr in to_impute:
    # boolean masks
    mask_znr  = panel["ZNR"] == znr
    mask_hist = panel["DATE"] <= pd.to_datetime("2024-12-01")
    mask      = mask_znr & mask_hist

    # extract and index by DATE
    tmp = (panel.loc[mask, ["DATE", "DTVMS"]]
              .set_index("DATE")
              .sort_index()
              .copy())

    # interpolate only inside known values
    tmp["DTVMS"] = tmp["DTVMS"].interpolate(
        method="time",
        limit_area="inside"
    )

    # forward/back-fill any head/tail NaNs
    tmp["DTVMS"] = tmp["DTVMS"].ffill().bfill()

    # write back into the main panel
    panel.loc[mask, "DTVMS"] = tmp["DTVMS"].values

# 4) (Optional) Verify that no gaps remain in history
def count_missing(grp):
    first = grp["DATE"].min()
    expected = pd.date_range(first, "2024-12-01", freq="MS")
    actual   = grp.loc[grp["DTVMS"] > 0, "DATE"].drop_duplicates().sort_values()
    return len(expected) - actual.isin(expected).sum()

gap_check = panel.groupby("ZNR").apply(count_missing).reset_index(name="months_missing_after")
print(gap_check[gap_check["ZNR"].isin(to_impute)])

     ZNR  months_missing_after
23  1199                     0
38  1219                     0
51  1618                     0


C:\Users\mihim\AppData\Local\Temp\ipykernel_143416\2901769923.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gap_check = panel.groupby("ZNR").apply(count_missing).reset_index(name="months_missing_after")


### 6) SAVE DATA

In [18]:
panel.to_csv("merged_df.csv", index=False)

In [19]:
df = pd.read_csv("merged_df.csv", parse_dates=["DATE"])